<center><h1>Zaimplementuj system sprawdzania typów</h1></center>

In [ ]:
class Descriptor():
    def __init__(self, name=None, **options):
        self.name = name
        for key, value in options.items():
            setattr(self, key, value)
            
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
class Typed(Descriptor):
    expected_type = type(None)
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError("Expected type: " + str(self.expected_type) + "but was: " + str(type(value)))
        super().__set__(instance, value)
        
class Unsigned(Descriptor):
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Expected value to be higher than 0, but was: " + str(value))
        super().__set__(instance, value)

In [ ]:
class MaxSize(Descriptor):
    def __init__(self, name=None, **options):
        if "max_size" not in options:
            raise TypeError("Expected max_size option")
        super().__init__(name, **options)

    def __set__(self, instance, value):
        if len(value) > self.max_size:
            raise ValueError("Expected value to be max size of: {}  but was: {}".format(str(self.max_size),
                                                                                       str(len(value))))
        super().__set__(instance, value)
        
class MinSize(Descriptor):
    def __init__(self, name=None, **options):
        if "min_size" not in options:
            raise TypeError("Expected min_size option")
        super().__init__(name, **options)
        
    def __set__(self, instance, value):
        if len(value) < self.min_size:
            raise ValueError("Expected value to be min size of: " + str(self.min_size) + " but was: " + str(len(value)))
        super().__set__(instance, value)

In [ ]:
class Integer(Typed):
    expected_type = int
    
class UnsignedInt(Integer, Unsigned): pass

class Float(Typed):
    expected_type = float
    
class UnsignedFloat(Float, Unsigned): pass

class String(Typed):
    expected_type = str
    
class MaxMinString(String, MaxSize, MinSize): pass

class List(Typed):
    expected_type = list
    
class MaxMinList(List, MaxSize, MinSize): pass

In [ ]:
class CustomStruct():
    x = Float("x")
    y = Integer("y")
    data_vector = MaxMinList("data_vector", min_size=2, max_size=10)
    
    def __init__(self, x, y, data_vector):
        self.x = x
        self.y = y
        self.data_vector = data_vector
        
cs = CustomStruct(1.0, 3, [1, 2, 3])
print(vars(cs))
cs.data_vector = list(range(7))
print(cs.data_vector)

In [ ]:
def typeassert(**kwargs):
    def type_assert_wrapper(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                print(key)
                value.name = key
                setattr(cls, key, value)
            else:
                print(key, value)
                setattr(cls, key, value(key))
        return cls
    return type_assert_wrapper

In [ ]:
@typeassert(
x=Float,
y=Integer,
data_vector=MaxMinList(max_size=5, min_size=1))
class PointVector():
    def __init__(self, x, y, data_vector):
        self.x = x
        self.y = y
        self.data_vector = data_vector
        
p = PointVector(1.0, 2, [1])
print(vars(p))
p.data_vector = [1, 2, 3, 4, 5]